this notebook is how we are cleaning the data based on the EDA we have previously done

First step in cleaning the data will be to remove all HTML tags in the text

In [1]:
import pandas as pd
import re
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [2]:
df=pd.read_csv("imdb_dataset.csv")

In [3]:
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

df['review']=df['review'].apply(lambda z: remove_punctuations(z))

df['review']=df['review'].apply(lambda z: remove_html(z))
df['review']=df['review'].apply(lambda z: remove_url(z))
df['review']=df['review'].apply(lambda z: remove_emoji(z))

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production br br The filmin...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [5]:
df.drop_duplicates(subset='review', inplace=True)


In [6]:
df.shape

(49581, 2)

In [7]:
# Check class distribution (optional)
print(df['sentiment'].value_counts())

# Get 1000 positive and 1000 negative reviews
positive_subset = df[df['sentiment'] == 'positive'].sample(n=1000, random_state=42)
negative_subset = df[df['sentiment'] == 'negative'].sample(n=1000, random_state=42)

# Combine and shuffle
balanced_subset = pd.concat([positive_subset, negative_subset]).sample(frac=1, random_state=42).reset_index(drop=True)


sentiment
positive    24884
negative    24697
Name: count, dtype: int64


In [8]:
balanced_subset.to_csv("balanced_subset_2000.csv", index=False)


In [9]:
import os

base_path = "/Users/yasminebenmessaoud/Library/CloudStorage/GoogleDrive-yasminebenme@gmail.com/.shortcut-targets-by-id/1Jmv-bun_bnRP4OD_2k7XcFpGWYLG2R4Z/NLP/output_trainer"
print(os.listdir(base_path))


['checkpoint-2500', 'checkpoint-4000', 'checkpoint-500', 'checkpoint-3500', 'checkpoint-1000', 'checkpoint-4704', 'checkpoint-4500', 'checkpoint-2000', 'checkpoint-1500', 'runs', 'checkpoint-3000']


In [10]:
checkpoint_path = "/Users/yasminebenmessaoud/Library/CloudStorage/GoogleDrive-yasminebenme@gmail.com/.shortcut-targets-by-id/1Jmv-bun_bnRP4OD_2k7XcFpGWYLG2R4Z/NLP/output_trainer/checkpoint-1000"


In [11]:
# Load model from local checkpoint
model = AutoModelForTokenClassification.from_pretrained(checkpoint_path, local_files_only=True)

# Load tokenizer from original base model (NOT from checkpoint!)
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2", use_fast=True)


In [12]:
from datasets import Dataset

# Tokenize into word tokens
df['tokens'] = df['review'].apply(lambda x: x.split())  # if not already split

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df[['tokens']].rename(columns={"tokens": "sents"}))


In [15]:
def tokenize_for_inference(examples):
    tokenized_inputs = tokenizer(
        examples["sents"],
        max_length=128,
        padding=False,
        truncation=True,
        is_split_into_words=True
    )
    return tokenized_inputs


In [16]:
tokenized_dataset = hf_dataset.map(tokenize_for_inference, batched=True)


Map:   0%|          | 0/49581 [00:00<?, ? examples/s]

AssertionError: You need to instantiate RobertaTokenizerFast with add_prefix_space=True to use it with pretokenized inputs.

In [14]:
import torch
import numpy as np

# Put model in evaluation mode
model.eval()

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

predicted_labels = []

for i in range(len(tokenized_dataset)):
    inputs = {k: v.unsqueeze(0).to(device) for k, v in tokenized_dataset[i].items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()[0]

    word_ids = tokenized_dataset[i]["input_ids"]  # raw tokens
    attention_mask = tokenized_dataset[i]["attention_mask"]
    
    # Decode only the relevant predicted token labels
    pred_labels = [idx2lab[pred] for pred, mask in zip(predictions, attention_mask) if mask == 1]
    predicted_labels.append(pred_labels)


NameError: name 'tokenized_dataset' is not defined

In [ ]:
df["predicted_labels"] = predicted_labels
df.head()
